---

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append(sys.path[0].replace('notebooks', 'src'))

import modeling.modeling_utils as m
from modeling.unbiasedemo_utils import make_df

In [40]:
m.torch.__version__

'1.6.0'

In [41]:
m.torch.cuda.is_available() # Check for GPU

False

In [42]:
device = m.torch.device("cuda" if m.torch.cuda.is_available() else "cpu")

In [43]:
m.torch.backends.cudnn.benchmark=True # Helps optimize training w/ GPU

---

This notebook tests 3 models on the ```UnBiasedEmo``` dataset. This dataset was specifically built to evaluate visual sentiment models, using the same objects/scenes (e.g., athletes, dogs) across different emotions (e.g., anger, joy). Instructions for downloading the dataset can be found here: https://rpand002.github.io/emotion.html. After downloading, unzip into the ```data``` directory.
<br>

The 3 models tested are:
1. The curriculum-based webemo_
2. The Deep Emotion fine-tuned Resnet50
3. A Resnet50 trained on ImageNet

# Loading Data

In [ ]:
deepemo_df = make_df('../data/Flickr')
deepemo_df.head()

Following the instructions above should give you a dataframe in this format:
```
                file    | labels
817 | joy/family/354.jpg | 4
1301 | love/cat/185.jpg | 2
1737 | love/scenery/176.jpg | 2
2783 | surprise/people/12 6.jpg | 0
435 | anger/tiger/68.jpg | 3
```

Next, I split the data into train/validation/test sets and utilize the dataset/dataloader classes for multiprocessing during training.

In [ ]:
train, test = m.train_val_split(unbiasedemo_df['file'], unbiasedemo_df['labels'], 
                                    test_size = 0.2, random_state=713)
train_split, val_split = train_val_split(train_df['file'], train_df['labels'], test_size = 0.1, random_state=611)

In [ ]:
train_transforms = m.transforms.Compose([m.transforms.Resize(256),
                                         m.transforms.RandomCrop(224),
                                         m.transforms.ToTensor(),
                                         m.transforms.Normalize(mean=[0.485, 0.456, 0.406], #OG means/sds from imagenet
                                                                std=[0.229, 0.224, 0.225])
                                        ])
val_transforms = m.transforms.Compose([m.transforms.Resize(256),
                                       m.transforms.CenterCrop(224),
                                       m.transforms.ToTensor(),
                                       m.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])
                                      ])

In [32]:
train_dataset = m.ImgDataset(df=train_split,
                                root_dir='../data/UnBiasedEmo/images',
                                percent_sample=1,
                                transform=train_transforms)
val_dataset = m.ImgDataset(df=val_split,
                              root_dir='../data/UnBiasedEmo/images',
                              percent_sample=1,
                              transform=val_transforms)

In [77]:
samples_weights = m.weighted_sample(train_split, 'labels')
weighted_sampler = m.WeightedRandomSampler(weights=samples_weights, num_samples=len(samples_weights))

In [35]:
loaders = {'train': m.DataLoader(train_dataset,
                               batch_size=128, 
                               sampler=weighted_sampler,
                               #pin_memory=True, # Only use pin_memory with GPU
                               num_workers=4), 
              'val': m.DataLoader(val_dataset, 
                             batch_size=128,
                             #pin_memory=True,
                             num_workers=4)}

# Training Models

### WEBEmo

In [ ]:
webemo_model = m.load_model(path='../models/l3model.tar', 
                        base=m.models.resnet50(pretrained=False), 
                        old_classes=25, 
                        new_classes=25, 
                        device=device)
# Freeze layers for feature extraction
for param in webemo_model.parameters():
    param.requires_grad = False
    
webemo_model.fc = m.nn.Sequential(nn.Linear(num_ftrs, 6))
webemo_model = webemo_model.to(device)
webemo_criterion = m.nn.CrossEntropyLoss().to(device)
webemo_optimizer = m.torch.optim.SGD(webemo_model.parameters(), 
                                  lr=0.001, 
                                  momentum=0.9, 
                                  weight_decay=0.0001)
webemo_scheduler = m.lr_scheduler.ReduceLROnPlateau(webemo_optimizer, patience=5, verbose=True)

In [ ]:
webemo_train = m.train_model(model=webemo_model, 
                             dataloader=loaders, 
                             criterion= webemo_criterion, 
                             optimizer= webemo_optim,
                             save_path='../models/webemo_exp.tar',
                             num_epochs=50,
                             scheduler= webemo_scheduler,
                             early_stopping=m.EarlyStopping(patience=10))

### Deep Emotion

In [ ]:
webemo_model = m.load_model(path='../models/deepemo_model.tar', 
                        base=m.models.resnet50(pretrained=False), 
                        old_classes=8, 
                        new_classes=8, 
                        device=device)

for param in deepemo_model.parameters():
    param.requires_grad = False
    
deepemo_model.fc = m.nn.Sequential(nn.Linear(2048, 6))
deepemo_model = deepemo_model.to(device)
deepemo_criterion = m.nn.CrossEntropyLoss().to(device)
deepemo_optimizer = m.torch.optim.SGD(deepemo_model.parameters(), 
                                  lr=0.001, 
                                  momentum=0.9, 
                                  weight_decay=0.0001)
deepemo_scheduler = m.lr_scheduler.ReduceLROnPlateau(deepemo_optimizer, patience=5, verbose=True)

In [ ]:
deepemo_train = m.train_model(model=deepemo_model, 
                             dataloader=loaders, 
                             criterion= deepemo_criterion, 
                             optimizer= deepemo_optim,
                             save_path='../models/deepemo_exp.tar',
                             num_epochs=50,
                             scheduler= deepemo_scheduler,
                             early_stopping=m.EarlyStopping(patience=10))

### ImageNet

In [ ]:
imagenet_model = models.resnet50(pretrained=True)

for param in imagenet_model.parameters():
    param.requires_grad = False
    
imagenet_model.fc = m.nn.Sequential(nn.Linear(2048, 6))
imagenet_model = imagenet_model.to(device)
imagenet_criterion = m.nn.CrossEntropyLoss().to(device)
imagenet_optimizer = m.torch.optim.SGD(imagenet_model.parameters(), 
                                  lr=0.001, 
                                  momentum=0.9, 
                                  weight_decay=0.0001)
imagenet_scheduler = m.lr_scheduler.ReduceLROnPlateau(imagenet_optimizer, patience=5, verbose=True)

In [ ]:
imagenet_train = m.train_model(model=imagenet_model, 
                             dataloader=loaders, 
                             criterion= imagenet_criterion, 
                             optimizer= imagenet_optim,
                             save_path='../models/imagenet_exp.tar',
                             num_epochs=50,
                             scheduler= imagenet_scheduler,
                             early_stopping=m.EarlyStopping(patience=10))

# Testing Models

In [ ]:
test_dataset = ImgDataset(df=test,
                           root_dir='../data/UnBiasedEmo/images',
                           percent_sample=1,
                           transform=val_transforms)
test_loader = DataLoader(test_dataset, 
                        batch_size=128,
                        #pin_memory=True,
                        num_workers=4)

### WEBEmo

In [ ]:
test_webemo_model = m.load_model(path='../models/webemo_exp.tar', 
                          base=m.models.resnet50(pretrained=False), 
                          old_classes=8, 
                          new_classes=8, 
                          device=device)
test_webemo_model = test_webemo_model.eval()
test_webemo_model = test_webemo_model.to(device)

In [ ]:
m.evaluate_model(test_webemo_model, test_loader)

```
tensor(0.7274, device='cuda:0', dtype=torch.float64)
```

### Deep Emotion

In [ ]:
test_deepemo_model = m.load_model(path='../models/deepemo_exp.tar', 
                          base=m.models.resnet50(pretrained=False), 
                          old_classes=8, 
                          new_classes=8, 
                          device=device)
test_deepemo_model = test_deepemo_model.eval()
test_deepemo_model = test_deepemo_model.to(device)

In [ ]:
m.evaluate_model(test_deepemo_model, test_loader)

```
tensor(0.6585, device='cuda:0', dtype=torch.float64)
```

### ImageNet

In [ ]:
test_imagenet_model = m.load_model(path='../models/imagenet_exp.tar', 
                          base=m.models.resnet50(pretrained=False), 
                          old_classes=8, 
                          new_classes=8, 
                          device=device)
test_imagenet_model = test_imagenet_model.eval()
test_imagenet_model = test_imagenet_model.to(device)

In [ ]:
m.evaluate_model(test_imagenet_model, test_loader)

```
tensor(0.5928, device='cuda:0', dtype=torch.float64)
```